# Analyze users by inspecting retweet counts

In [12]:
import pandas as pd
from IPython.display import clear_output

In [13]:
DATADIR = "/home/erikt/projects/puregome/data/retweets/"
DATAFILE = "get-retweet-users.py.out.gz"
RIGHT = "geertwilderspvv thierrybaudet fvdemocratie".split(" ")
DUTCHPRESS = "telegraaf NOS RTLnieuws NUnl ADnl nrc volkskrant trouw parool AT5 NPORadio1 Nieuwsuur op1npo EenVandaag".split(" ")
FLEMISHPRESS = "HLN_BE demorgen destandaard vrtnws Nieuwsblad_be deafspraaktv VTMNIEUWS".split(" ")

In [14]:
def squeal(text=None):
    clear_output(wait=True)
    if not text is None: print(text)

## Read the retweet data

In [15]:
df = pd.read_csv(DATADIR+DATAFILE,sep=" ",header=None,compression="gzip")

## Create a lists of retweeted users per retweeter

In [16]:
groups = df.groupby([0,1]).groups

## Check how often each retweeter retweets users of the three main groups 

In [17]:
profiles = {}

def add(user,group,count):
    if not user in profiles: profiles[user] = {RIGHT[0]:0,DUTCHPRESS[0]:0,FLEMISHPRESS[0]:0}
    if not group[0] in profiles[user]: profiles[user][group[0]] = count
    else: profiles[user][group[0]] += count

counter = 0
for key in groups.keys():
    counter += 1
    if counter % 10000 == 0: squeal(str(counter)+"/"+str(len(groups)))
    retweetedUser = key[1]
    retweeter = key[0]
    if retweetedUser in RIGHT: add(retweeter,RIGHT,len(groups[key]))
    elif retweetedUser in DUTCHPRESS: add(retweeter,DUTCHPRESS,len(groups[key]))
    elif retweetedUser in FLEMISHPRESS: add(retweeter,FLEMISHPRESS,len(groups[key]))

3450000/3458767


## Get users with retweets of only one group

In [ ]:
MIN = 5
usersPerGroup = {RIGHT[0]:[],DUTCHPRESS[0]:[],FLEMISHPRESS[0]:[]}
for user in profiles:
    if profiles[user][RIGHT[0]] > MIN and profiles[user][DUTCHPRESS[0]] == 0 and profiles[user][FLEMISHPRESS[0]] == 0:
        usersPerGroup[RIGHT[0]].append(user)
    if profiles[user][RIGHT[0]] == 0 and profiles[user][DUTCHPRESS[0]] > MIN and profiles[user][FLEMISHPRESS[0]] == 0:
        usersPerGroup[DUTCHPRESS[0]].append(user)
    if profiles[user][RIGHT[0]] == 0 and profiles[user][DUTCHPRESS[0]] == 0 and profiles[user][FLEMISHPRESS[0]] > MIN:
        usersPerGroup[FLEMISHPRESS[0]].append(user)

In [ ]:
len(usersPerGroup[RIGHT[0]]),len(usersPerGroup[DUTCHPRESS[0]]),len(usersPerGroup[FLEMISHPRESS[0]])

## Check for other interesting retweeted users per group (optional)

In [ ]:
retweetedUsers = []
counter = 0
for user in usersPerGroup[RIGHT[0]]:
    counter += 1
    squeal(str(counter)+"/"+str(len(usersPerGroup[RIGHT[0]])))
    retweetedUsers.extend([ x[1] for x in groups if x[0] == user ])
retweetedUsersDict  = pd.DataFrame(retweetedUsers).groupby(0).groups

In [ ]:
list({key:len(retweetedUsersDict[key]) for key in sorted(retweetedUsersDict.keys(),
                                                         key=lambda k:len(retweetedUsersDict[k]),reverse=True)}.items())[0:20]

In [ ]:
list({key:len(retweetedUsersDict[key]) for key in sorted(retweetedUsersDict.keys(),
                                                         key=lambda k:len(retweetedUsersDict[k]),reverse=True)}.items())[0:20]

In [ ]:
list({key:len(retweetedUsersDict[key]) for key in sorted(retweetedUsersDict.keys(),
                                                         key=lambda k:len(retweetedUsersDict[k]),reverse=True)}.items())[0:20]

## Get tweet texts of selected users (optional)

In [ ]:
import os
import re

In [ ]:
DATADIRTEXT = "/home/erikt/projects/puregome/data/text/"
FILEPATTERN = "2020060[1-7]"
USER = "user"
TEXT = "text"

In [ ]:
selectedUsers = usersPerGroup[RIGHT[0]]+usersPerGroup[DUTCHPRESS[0]]+usersPerGroup[FLEMISHPRESS[0]]
files = sorted(os.listdir(DATADIRTEXT))
tweetTexts = {}
for inFileName in files:
    if re.search(FILEPATTERN,inFileName):
        squeal(inFileName)
        df = pd.read_csv(DATADIRTEXT+inFileName)
        for i in range(0,len(df)):
            user = df.iloc[i][USER]
            if df.iloc[i][USER] in selectedUsers:
                if not user in tweetTexts: tweetTexts[user] = df.iloc[i][TEXT]
                else: tweetTexts[user] += " "+df.iloc[i][TEXT]
tweetTexts

## Tokenize tweet texts (optional)

In [ ]:
from nltk.tokenize import TweetTokenizer

In [ ]:
def removeKeywords(text):
    for keyword in RIGHT+DUTCHPRESS+FLEMISHPRESS:
        text = re.sub(r"\b"+keyword.lower()+r"\b"," ",text)
    return(text)

In [ ]:
tokenizedTweetTexts = {}
tokenizer = TweetTokenizer()
for user in tweetTexts.keys():
    tokenizedTweetTexts[user] = " ".join(tokenizer.tokenize(re.sub(r"\\n"," ",removeKeywords(tweetTexts[user].lower()))))

## Save tweet texts (optional)

In [ ]:
OUTFILENAME = "groups-train.txt"
LABELPREFIX = "__label__"
RIGHTTEXT = "RIGHT"
DUTCHPRESSTEXT = "DUTCHPRESS"
FLEMISHPRESSTEXT = "FLEMISHPRESS"

outFile = open(OUTFILENAME,"w")
for user in tokenizedTweetTexts:
    if user in usersPerGroup[RIGHT[0]]: label = RIGHTTEXT
    elif user in usersPerGroup[DUTCHPRESS[0]]: label = DUTCHPRESSTEXT
    elif user in usersPerGroup[FLEMISHPRESS[0]]: label = FLEMISHPRESSTEXT
    else: sys.exit("cannot happen!")
    print(LABELPREFIX+label,tokenizedTweetTexts[user],file=outFile)
outFile.close()

## Evaluate fasttext on data (optional)

In [ ]:
import fasttext
import os
import random

In [ ]:
TRAINFILE = "groups-train.txt"
LARGEINT = 99999999

In [ ]:
fasttextData = []
inFile = open(TRAINFILE,"r")
for line in inFile: fasttextData.append(line.strip())
inFile.close()

In [ ]:
DIM = 300
EPOCH = 100
LR = 0.1
N = 10
TRAIN = "TRAIN"+str(int(random.random()*LARGEINT))
TEST = "TEST"+str(int(random.random()*LARGEINT))

predictionCounts = []
predictionLabels = []
for fold in range(0,N):
    squeal("starting fold "+str(fold))
    testStart = round(fold*len(fasttextData)/N)
    testEnd = round((fold+1)*len(fasttextData)/N)
    trainFile = open(TRAIN,"w")
    testFile = open(TEST,"w")
    testData = []
    for i in range(0,len(fasttextData)):
        if i < testStart or i >= testEnd: print(fasttextData[i],file=trainFile)
        else: 
            print(fasttextData[i],file=testFile)
            testData.append(fasttextData[i])
    testFile.close()
    trainFile.close()
    model = fasttext.train_supervised(TRAIN,dim=DIM,epoch=EPOCH,lr=LR) # ,pretrainedVectors=PRETRAINEDDIR+WIKIFILENAME)
    predictionCounts.append([*model.test(TEST)])
    predictionLabels.append(model.predict(testData))
    os.unlink(TRAIN)
    os.unlink(TEST)
clear_output(wait=True)
print("finished")

In [ ]:
caseTotal = 0
pTotal = 0
rTotal = 0
for i in range(0,len(predictionCounts)):
    caseTotal += predictionCounts[i][0]
    pTotal += predictionCounts[i][0]*predictionCounts[i][1]
    rTotal += predictionCounts[i][0]*predictionCounts[i][2]
print("cases: {0}; precision: {1}; recall: {2}".format(caseTotal,round(pTotal/caseTotal,3),round(rTotal/caseTotal,3)))

Scores: epoch=10 P=52; epoch=20 P=86; epoch=30 P=89; epoch=50 P=91; epoch=100 P=94.7; epoch=200 P=94.8; epoch=130 P=95.1; epoch=150 P=95.0

In [ ]:
labelCountsGold = {}
for i in range(0,len(fasttextData)):
    label = fasttextData[i].split()[0]
    if label in labelCountsGold: labelCountsGold[label] += 1
    else: labelCountsGold[label] = 1
labelCountsGold

In [ ]:
labelCountsPredicted = {}
for i in range(0,len(predictionLabels)):
    for label in predictionLabels[i][0]:
        if label[0] in labelCountsPredicted: labelCountsPredicted[label[0]] += 1
        else: labelCountsPredicted[label[0]] = 1
labelCountsPredicted

In [ ]:
for i in range(0,len(fasttextData)):
    labelGold = fasttextData[i].split()[0]
    labelPredicted = predictionLabels[i][0]
    print(labelGold,labelPredicted)
    break

## Apply fasttext to other data

In [44]:
import csv
import fasttext
import os
import re
import pandas as pd
from nltk.tokenize import TweetTokenizer
from IPython.display import clear_output

In [2]:
TRAINFILE = "groups-train.txt"
DIM = 300
EPOCH = 100
LR = 0.1
RIGHTTEXT = "RIGHT"
DUTCHPRESSTEXT = "DUTCHPRESS"
FLEMISHPRESSTEXT = "FLEMISHPRESS"

In [ ]:
def squeal(text=None):
    clear_output(wait=True)
    if not text is None: print(text)

In [3]:
model = fasttext.train_supervised(TRAINFILE,dim=DIM,epoch=EPOCH,lr=LR)

In [31]:
TEXT = "text"
USER = "user"
TESTFILEPATTERN = "2020061[6-9]|2020062[0-2]"
DATADIRTEXT = "/home/erikt/projects/puregome/data/text/"

files = sorted(os.listdir(DATADIRTEXT))
tokenizer = TweetTokenizer()
userTexts = {}
for inFileName in files:
    if re.search(TESTFILEPATTERN,inFileName):
        squeal(inFileName)
        df = pd.read_csv(DATADIRTEXT+inFileName)
        for i in range(0,len(df)): 
            user = df.iloc[i][USER]
            text = df.iloc[i][TEXT]
            if user in userTexts: userTexts[user] += " "+text
            else: userTexts[user] = text
tokenizedTexts = {}
for user in userTexts:
    tokenizedTexts[user] = " ".join(tokenizer.tokenize(re.sub(r"\\n"," ",userTexts[user].lower())))
predictedLabels = model.predict([tokenizedTexts[user] for user in tokenizedTexts.keys()])

20200622-23.out.gz


In [36]:
LABEL = "label"
CONFIDENCE = "confidence"

predictedLabelsDict = {}
counter = 0
for user in tokenizedTexts:
    predictedLabelsDict[user] = { LABEL:predictedLabels[0][counter][0], CONFIDENCE:predictedLabels[1][counter][0] }
    counter += 1

In [49]:
MAXCOUNTER = 1

counter = 0
for user in sorted(tokenizedTexts.keys(),key=lambda k:len(tokenizedTexts[k]),reverse=True):
    print(predictedLabelsDict[user][LABEL],end=" ")
    print(predictedLabelsDict[user][CONFIDENCE],end=" ")
    if not user in profiles: print("???",end=" ")
    elif profiles[user][RIGHT[0]] > 0: print(RIGHTTEXT,end=" ")
    elif profiles[user][FLEMISHPRESS[0]] > 0: print(FLEMISHPRESSTEXT,end=" ")
    elif profiles[user][DUTCHPRESS[0]] > 0: print(DUTCHPRESSTEXT,end=" ")
    else: print("?!?",end=" ")
    print(tokenizedTexts[user][0:500])
    counter += 1
    if counter >= MAXCOUNTER: break

__label__DUTCHPRESS 0.8073545 ??? lees " hard rock hotel amsterdam american swingt " op nieuwsblik - https://t.co/uf3iz7hhpu - lees verder bij de bron van het artikel lees " luuk de jong in slotfase beroofd van heldenrol " op nieuwsblik - https://t.co/gyonwxvjmg - lees verder bij de bron van het artikel lees " champions league start op 8 augustus " op nieuwsblik - https://t.co/cwkkvjbfjm - lees verder bij de bron van het artikel lees " ‘ pompeo naar hawaii voor overleg met china ’ " op nieuwsblik - https://t.co/rge8fufq4w - de a


In [48]:
MAXCOUNTER = 5000
DATAFILEOUT = "user-groups.csv"
ID = "ID"

counter = 0
outFile = open(DATAFILEOUT,"w")
csvwriter = csv.DictWriter(outFile,[ID,USER,TEXT])
for user in sorted(tokenizedTexts.keys(),key=lambda k:len(tokenizedTexts[k]),reverse=True):
    csvwriter.writerow({ID:user,USER:user,TEXT:tokenizedTexts[user][0:1000]})
    counter += 1
    if counter >= MAXCOUNTER: break
outFile.close()